# Personnes en emploi travaillant à la maison, en pourcentage du total de l'emploi, par sexe, âge et statut professionnel (%) 

### Adresse web
- lien = https://data.europa.eu/euodp/fr/data/dataset/orJJzGDF3cnXimvsoKDfXw

### Fichier source
- de 1992 à 2019 : Statistiques sur le télétravail avec critères par pays Européens

### unite : 
    # PC = Pourcentage
------------------------------------------------------------------------------------------------------------------

### sexe : 
    # M = man
    # F = Femmme
    # T = Total
------------------------------------------------------------------------------------------------------------------
 
### frequence : 
    # NVR = never
    # USU = usually
    # SMT = sometimes
------------------------------------------------------------------------------------------------------------------
    
### statut : 
    # 'CFAM' = Contributing family workers
    # 'EMP' = Employed persons
    # 'NCFAM' = Employed persons except contributing family workers
    # 'NRP' = No response
    # 'NSAL' = Employed persons except employees
    # 'SAL' = Employees
    # 'SELF' = Self-employed persons
    # 'SELF_NS' = Self-employed persons without employees (own-account workers)
    # 'SELF_S' = Self-employed persons with employees (employers)
    
### statut (FR) : 
    # 'CFAM' = Travailleurs familiaux collaborant à l'entreprise familiale
    # 'EMP' = Personnes occupées
    # 'NCFAM' = Personnes occupées sauf travailleurs familiaux collaborant à l'entreprise familiale
    # 'NRP' = Sans réponse
    # 'NSAL' = Personnes occupées sauf salariés
    # 'SAL' = salarié
    # 'SELF' = Travailleurs indépendants
    # 'SELF_NS' = Travailleurs indépendants sans salariés
    # 'SELF_S' = Travailleurs indépendants avec salariés (employeurs)
 
------------------------------------------------------------------------------------------------------------------
    
### age : 
    # par intervalle
        # code : Y_GE15 = 15 years and over

## 1) Import des bibliothèques et paramètres

In [2]:
# Importation des bibliothèques
import pandas as pd
import numpy as np

# Paramétrage des options d'affichage des colonnes
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)

## 2) Nettoyage des données

In [3]:
# Définition de l'objet "datas"
datas = pd.read_csv("./lfsa_ehomp.tsv", sep='\t', engine='python')

# 1ère partie du dataset : sélection sous forme de "series"
datas_1 = datas.iloc[:,0]

# Conversion et split en dataframe avec colonnes
datas_1 = datas_1.str.split(',', expand=True)

# Renommage des colonnes
datas_1.rename(columns={0:'unite', 1:'sexe', 2:'frequence', 3:'age', 4:'statut_w', 5:'code_pays'}, inplace=True)

# 2ème partie du dataset
datas_2 = datas.iloc[:,1:]

# Suppression des lettres "u" et "b" dans le dataset
datas_3 = datas_2.replace({'u':''}, regex=True).replace({'b':''}, regex=True)


# Remplacement des valeurs inconnues
def remplacement():
    col = datas_3.columns
    for i in col:
        datas_3.loc[datas_3[i].str.contains(':', na=False), i] = 888   
    #datas_3 = datas_2
    return datas_3

# Concatenation des deux datasets (datas_1 & datas_3)
df = pd.concat([datas_1, remplacement()], axis=1)

# Chargement du dataset "pays" afin de traiter les abbréviations 
pays = pd.read_csv("./pays_2.csv", engine='python')

# Renommage des colonnes
pays.rename(columns={'Name':'nom_pays', 'Code': 'code'}, inplace=True)

# Merged dataframe avec nom des pays
df_2 = pd.merge(df, pays, how='left', left_on=['code_pays'], right_on = ['code']).drop("code", axis = 1)

# Remplacement des valeurs Nan dans la colonne "nom_pays"
df_2.loc[df_2['code_pays'] == "EL", 'nom_pays'] = "Greece"
df_2.loc[df_2['code_pays'] == "UK", 'nom_pays'] = "United Kingdom"
df_2.loc[df_2['code_pays'] == "EA19", 'nom_pays'] = "EA19"
df_2.loc[df_2['code_pays'] == "EU15", 'nom_pays'] = "EU15"
df_2.loc[df_2['code_pays'] == "EU27_2020", 'nom_pays'] = "EU27_2020"
df_2.loc[df_2['code_pays'] == "EU28", 'nom_pays'] = "EU28"

# Réindexation des colonnes pour placer "nom_pays" après "pays" (code pays)
column_names = ['unite', 'sexe', 'frequence', 'age', 'statut_w', 'code_pays', 'nom_pays', '2019 ',
       '2018 ', '2017 ', '2016 ', '2015 ', '2014 ', '2013 ', '2012 ', '2011 ',
       '2010 ', '2009 ', '2008 ', '2007 ', '2006 ', '2005 ', '2004 ', '2003 ',
       '2002 ', '2001 ', '2000 ', '1999 ', '1998 ', '1997 ', '1996 ', '1995 ',
       '1994 ', '1993 ', '1992 ']

df_2 = df_2.reindex(columns=column_names)

# Remplacement des lettres "T" par "total", "H" par "homme" et "F" par "femme" pour traitement sous Tableau
df_2["sexe"] = df_2["sexe"].replace('T','total').replace('M','men').replace('F','women')

In [6]:
df_2.shape

(70561, 35)

In [4]:
df_2.head()

,unite,sexe,frequence,age,statut_w,code_pays,nom_pays,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994,1993,1992
0,PC,women,NVR,Y15-19,CFAM,AT,Austria,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888
1,PC,women,NVR,Y15-19,CFAM,BE,Belgium,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888
2,PC,women,NVR,Y15-19,CFAM,BG,Bulgaria,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888
3,PC,women,NVR,Y15-19,CFAM,CH,Switzerland,91.5,91.3,76.6,888,91.1,62.3,79.0,91.8,88.6,93.1,75.0,79.4,56.7,95.6,69.8,97.3,68.3,84.6,888,67.3,56.3,56.4,75.7,888,888,888,888,888
4,PC,women,NVR,Y15-19,CFAM,CY,Cyprus,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888,888


# 3) Export

In [4]:
# Export du dataset créé au format CSV
df_2.to_csv(r'./lien3_dataset.csv')